In [ ]:
# load the csv with the data
import pandas as pd
import re
data = pd.read_csv('results_iso_check.csv')

In [ ]:
# Some config
MULTI_EDGE = False
model = "gpt-4"


In [ ]:
# select data, i.e. fixed_prompt column. select only rows where completion_type = ['Add_node']
count = 0
for index, row in data.iterrows():
    if row['completion_type'] == "['Add_node']" and re.match(r".*'changeType': 'Add', 'type': 'object'.*",row['completion']):
        count += 1

print(f"There are {count} rows with completion_type = ['Add_node']")

Run the completion with gpt-4

In [ ]:
# define the prompts

from GPT_runner import sample_edge_gpt_chat, CHAT_MODEL_INSTRUCTION

# select data, i.e. fixed_prompt column. select only rows where completion_type = ['Add_node']
count = 0
for index, row in data.iterrows():
    if row['completion_type'] == "['Add_node']" and re.match(r".*'changeType': 'Add', 'type': 'object'.*",row['completion']):
        total_tokens, completion_tokens, completion_string = sample_edge_gpt_chat(model, row['prompt'], CHAT_MODEL_INSTRUCTION, MULTI_EDGE)
        count += 1
        # set in data file
        data.at[index, 'completion_generated'] = completion_string
        data.at[index, 'completion_tokens'] = completion_tokens
        data.at[index, 'total_tokens'] = total_tokens

# save the data
data.to_csv('results_iso_check.csv', index=False)

